In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import nest_asyncio


sys.path.insert(0, os.path.abspath('..'))
nest_asyncio.apply()

In [ ]:
from decouple import config

from math_rag.infrastructure.repositories.documents import MongoDocumentRepository
from math_rag.infrastructure.repositories.embeddings import QdrantEmbeddingRepository
from math_rag.infrastructure.repositories.graphs import Neo4jGraphRepository
from math_rag.infrastructure.repositories.objects import MinioObjectRepository


DEPLOYMENT = config('DEPLOYMENT')

MINIO_ENDPOINT = config('MINIO_ENDPOINT')
MINIO_ACCESS_KEY = config('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = config('MINIO_SECRET_KEY')
MONGO_HOST = config('MONGO_HOST')
NEO4J_URI = config('NEO4J_URI')
NEO4J_USERNAME = config('NEO4J_USERNAME')
NEO4J_PASSWORD = config('NEO4J_PASSWORD')
QDRANT_URL = config('QDRANT_URL')

In [ ]:
file_repo = MinioObjectRepository(MINIO_ENDPOINT, MINIO_ACCESS_KEY, MINIO_SECRET_KEY)
embedding_repo = QdrantEmbeddingRepository(QDRANT_URL)
graph_repo = Neo4jGraphRepository(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
object_repo = MongoDocumentRepository(MONGO_HOST, DEPLOYMENT)

### New

In [ ]:
from math_rag.infrastructure.containers import InfrastructureContainer


infrastructure_container = InfrastructureContainer()
infrastructure_container.init_resources()
infrastructure_container.wire(
    modules=[__name__]
)  # NOTE not needed if using objects from container directly (without provide and inject)

In [ ]:
from dependency_injector.wiring import Provide, inject

from math_rag.infrastructure.seeders.objects import MathArticleSeeder


@inject
def run_seeder(
    seeder: MathArticleSeeder = Provide[InfrastructureContainer.math_article_seeder],
):
    seeder.seed()

In [ ]:
# NOTE this code doesnt require wiring
math_article_seeder = infrastructure_container.math_article_seeder()
math_article_seeder.seed()